In [2]:
%load_ext autoreload
%autoreload 2
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
# import torch.utils.data.distributed
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
# import torchvision.models as models

from torch.utils import data
import random
import numpy as np
from itertools import product

import torch.nn.functional as F

from args import args
from train_f import *
from Dataset import Dataset



In [131]:
pred = torch.Tensor(np.zeros((2,2,4,4,4)))
pred[:,0,:,:,:] = 99
#pred[0,1,1,1] = 1
pred[:,1,0,0,0] = 99
pred[:,0,0,0,0] = 0

pred[:,1,0,0,1] = 99
pred[:,0,0,0,1] = 0

pred[:,1,1,0,0] = 99
pred[:,0,1,0,0] = 0

pred[:,1,2,0,0] = 99
pred[:,0,2,0,0] = 0


pred

tensor([[[[[ 0.,  0., 99., 99.],
           [99., 99., 99., 99.],
           [99., 99., 99., 99.],
           [99., 99., 99., 99.]],

          [[ 0., 99., 99., 99.],
           [99., 99., 99., 99.],
           [99., 99., 99., 99.],
           [99., 99., 99., 99.]],

          [[ 0., 99., 99., 99.],
           [99., 99., 99., 99.],
           [99., 99., 99., 99.],
           [99., 99., 99., 99.]],

          [[99., 99., 99., 99.],
           [99., 99., 99., 99.],
           [99., 99., 99., 99.],
           [99., 99., 99., 99.]]],


         [[[99., 99.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]],

          [[99.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]],

          [[99.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]],

          [[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0., 

In [132]:
target= torch.Tensor(np.zeros((2,4,4,4))).long()
target[:,0,0,0] = 1
target

tensor([[[[1, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]],


        [[[1, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]]])

In [137]:
outputs = F.softmax(pred, dim=1)
predicted = outputs.max(1)[1]
predicted

tensor([[[[1, 1, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[1, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[1, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]],


        [[[1, 1, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[1, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[1, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]]])

In [134]:
criterion = nn.CrossEntropyLoss()
loss = criterion(pred, target)
loss

tensor(4.6406)

In [145]:
a = 

In [146]:
torch.prod(torch.Tensor(list(predicted.size())))

tensor(128.)

In [157]:
def blob_loss(predicted, kernel_size = 2):
    predict = predicted
    num_obs = predicted.size(0)
    length = predicted.size(1)
    cube_sums = []
    for i0 in range(num_obs):
        for i in range(0,length - kernel_size + 1):
            for j in range(0,length - kernel_size + 1):
                for k in range(0,length - kernel_size + 1):
    #                 print('i' + str(i))
    #                 print('j' + str(j))
    #                 print('k' + str(k))
                    cube_sum = torch.sum(predicted[i0,i:i + kernel_size, j:j + kernel_size,k:k + kernel_size])
                    cube_sum_1 = 0 if cube_sum < 2 else (cube_sum.item() - 1) **2
                    cube_sums.append(cube_sum_1)
                    #print(cube_sum_1)
    return torch.sum(torch.Tensor(cube_sums)) / torch.prod(torch.Tensor(list(predicted.size())))

In [158]:
criterion = nn.CrossEntropyLoss(weight = torch.Tensor([0.5,0.5]))
loss_nn = criterion(pred, target)
loss_blob = blob_loss(predicted, kernel_size = 2)
print(loss_nn)
print(loss_blob)
loss = loss_nn + torch.tensor(0.1 * loss_blob)
loss

tensor(4.6406)
tensor(0.0781)


tensor(4.6484)

In [166]:
def yqloss(weight, w):
    def yqloss_(pred, target, w = 5, kernel_size = 3):
        criterion = nn.CrossEntropyLoss(weight = weight)
        loss_nn = criterion(pred, target)
        loss_blob = blob_loss(predicted, kernel_size = kernel_size)
        loss = loss_nn + torch.tensor(w * loss_blob)
        return loss
    return yqloss_
criterion = yqloss(torch.Tensor([0.5,0.5]),torch.Tensor([10]))
criterion(pred, target)

tensor(5.4219)